In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

repo_dir = '../'

# Scoring dnSVs using SuPreMo-Akita

In [52]:
# Generate input for SuPreMo
dnSVs = pd.read_csv(f'{repo_dir}data/SFARI_SSC_dnSVs.csv').rename(columns = {'#chrom':'chrom'})
dnSVs_for_supremo = dnSVs[[x in ['DEL', 'DUP', 'INV'] for x in dnSVs.svtype]][['chrom', 'pos', 'end', 'svtype', 'svlen']].drop_duplicates()
dnSVs_for_supremo.insert(2, 'REF', '-')
dnSVs_for_supremo.insert(3, 'ALT', '-')
dnSVs_for_supremo.columns = ['CHROM', 'POS', 'REF', 'ALT', 'END', 'SVTYPE', 'SVLEN']
dnSVs_for_supremo.to_csv(f'{repo_dir}data/dnSVs_for_SuPreMo.txt', sep = '\t', index = False)

In [55]:
# Read SuPreMo output and save to file

supremo_scores = pd.concat([pd.read_csv(f'{repo_dir}data/dnSVs_for_SuPreMo.txt', 
                                        sep = '\t'),
                            pd.read_csv(f'{repo_dir}scoring_dnSVs/supremo-akita_output/SSC_dnSV_scores', 
                                        sep = '\t')],
                           axis = 1)

supremo_scores['corr_median'] = [np.nanmedian([x,y,z]) for x,y,z in zip(supremo_scores['corr_-1'], 
                                                                         supremo_scores['corr_0'], 
                                                                         supremo_scores['corr_0_revcomp'])]
supremo_scores['mse_median'] = [np.nanmedian([x,y,z]) for x,y,z in zip(supremo_scores['mse_-1'], 
                                                                         supremo_scores['mse_0'], 
                                                                         supremo_scores['mse_0_revcomp'])]

supremo_scores.columns = ['chrom', 'pos', 'REF', 'ALT', 'end', 'svtype', 'svlen', 'var_index',
       'mse_-1', 'corr_-1', 'mse_0', 'corr_0', 'mse_0_revcomp',
       'corr_0_revcomp', 'mse_1', 'corr_1', 'corr_median', 'mse_median']

SSC_results_supremo = (dnSVs
                       .rename(columns = {'#chrom':'chrom'})
                       .merge(supremo_scores.drop(['REF', 'ALT'], axis = 1),
                              on = ['chrom', 'pos', 'end', 'svtype', 'svlen'], how = 'left'))

SSC_results_supremo.to_csv(f'{repo_dir}data/SFARI_SSC_dnSVs', sep = '\t', index = False)                                                               

/pollard/home/ketringjoni/miniconda3/envs/get_Akita_scores_env/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/pollard/home/ketringjoni/miniconda3/envs/get_Akita_scores_env/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


# Scoring dnSVs near CREints using SuPreMo-Akita with weighted scoring

## Process PLACseq data into CREints for weighting

### Get data

In [6]:
from pybedtools import BedTool

In [10]:
# Get hg38 promoters of protein-coding genes (Kallisto https://pachterlab.github.io/kallisto/)

ensembl = pd.read_csv(f'{repo_dir}data/Homo_sapiens.GRCh38.96.gtf',
                     sep='\t', skiprows=5, header=None, 
                      names=['chrom','source','desc','start','end','score','strand','score2','desc_further'],
                     low_memory=False)
ensembl['chr'] = 'chr' + ensembl['chrom'].astype(str)
ensembl['name'] = ensembl['desc_further'].str.split('gene_name ').str[1].str.split(';').str[0].str.strip('""')
ensembl['biotype'] = ensembl['desc_further'].str.split('biotype ').str[1].str.split(';').str[0].str.strip('""')

# Remove nonsense chromosomes
chromosomes = []
for i in list(range(23))[1:]:
    chrom = 'chr' + str(i)
    chromosomes.append(chrom)
chromosomes.append('chrX')
chromosomes.append('chrY')
ensembl = ensembl[[x in chromosomes for x in ensembl.chr]]

ensembl = ensembl[ensembl.biotype == "protein_coding"]
ensembl_genes = ensembl.query('desc == "gene"').copy()
n_ens_genes = pd.DataFrame(ensembl_genes['name'].value_counts())
n_ens_genes['gene'] = n_ens_genes.index
ensembl_genes_single = n_ens_genes.query('name == 1')['gene'].tolist()
ensembl_genes_single_df = ensembl_genes.query('(name in @ensembl_genes_single)').copy()

# Get ensemble gene ids
ensembl_genes_single_df['gene_id'] = ensembl_genes_single_df['desc_further'].str.split('"').str[1]
ensembl_genes_single_df = ensembl_genes_single_df.rename(columns = {"name":"gene"})
gene_annot = ensembl_genes_single_df[['gene', 'gene_id', 'chr', 'start', 'end', 'strand']]

promoter_annot = gene_annot.copy()

def label_promoter_start(row):
    if row['strand'] == '+' :
        return row['start'] - 1000
    if row['strand'] == '-' :
        return row['end']

promoter_annot['Start'] = promoter_annot.apply(lambda row: label_promoter_start(row), axis=1)
promoter_annot['Start'][promoter_annot['Start'] < 1] = 1
promoter_annot['End'] = promoter_annot['Start'] + 1000

promoter_annot_PC_BED = BedTool.from_dataframe(promoter_annot[['chr', 'Start', 'End', 'gene']])

In [3]:
# Get excitatory neuron H3K4me3 PLACseq loops (Song et al 2021)

loops = pd.read_csv(f'{repo_dir}data/eN.MAPS.peaks.txt', sep = '\t').rename(columns = {'chr1':'chrom'})[['chrom', 'start1', 'end1', 'start2', 'end2']]

# Get loops within predictive window 

dist_cutoff = 900000 #pixel_size*448

center_coord1 = [(x+y)/2 for x,y in zip(loops.start1, loops.end1)]
center_coord2 = [(x+y)/2 for x,y in zip(loops.start2, loops.end2)]

before = len(loops)

distance = [abs(x-y) for x,y in zip(center_coord1, center_coord2)]
loops = loops[[x < dist_cutoff for x in distance]]

print(len(loops)/before*100, '% of loops are within predictive window')


97.77350192413414 % of loops are within predictive window


In [36]:
# Get excitatory neuron RNAseq data (Song et al 2021)

eN_exp = pd.read_csv(f'{repo_dir}data/eN_avg', sep = '\t').dropna() 

In [40]:
# Get excitatory neuron ATACseq data (Song et al 2021)

ATAC = pd.read_csv(f'{repo_dir}data/eN.ATAC-seq.narrowPeak', sep = '\t', 
                   names = ['chrom', 'start', 'end', 'peak_id', 'score', 'strand', 'signalValue', 'pvalue', 'qvalue', 'peak'])
ATAC_BED = BedTool.from_dataframe(ATAC[['chrom', 'start', 'end']]).sort()

### Grop nearby loop anchors

In [4]:
# Get left and right loops

left_loops = loops[['chrom', 'start1', 'end1']].drop_duplicates().reset_index(drop = True)
left_loops['left_index'] = left_loops.index
loops = loops.merge(left_loops, on = ['chrom', 'start1', 'end1'])

right_loops = loops[['chrom', 'start2', 'end2']].drop_duplicates().reset_index(drop = True)
right_loops['right_index'] = right_loops.index
loops = loops.merge(right_loops, on = ['chrom', 'start2', 'end2'])

In [7]:
# For all loops with the same left anchor, combine right anchors that are within d

d = 10000

for i in left_loops.left_index:
    
    loops_i = loops.query('left_index == @i')[['chrom', 'start2', 'end2', 'right_index']]
    right_loop_BED = BedTool.from_dataframe(loops_i)

    if len(right_loop_BED) > 1:
        right_loop_keep = (right_loop_BED
                           .sort()
                           .merge(d = d, c = 4, o = 'collapse') # collapse indexes of the anchors merged
                           .to_dataframe()
                           .rename(columns = {'name':'right_index'}))
        
        if len(right_loop_keep) < len(loops_i):
            
            right_loop_keep['right_index'] = right_loop_keep['right_index'].str.split(',')
            right_loop_keep = right_loop_keep.explode('right_index')

            for ii in right_loop_keep.right_index.unique():

                loops.loc[(loops.left_index == i) & 
                          (loops.right_index == int(ii)),
                          'start2b'] = int(right_loop_keep.loc[right_loop_keep.right_index == ii,'start'])
                loops.loc[(loops.left_index == i) & 
                          (loops.right_index == int(ii)),
                          'end2b'] = int(right_loop_keep.loc[right_loop_keep.right_index == ii,'end'])

In [11]:
# Repeat for the right anchor

for i in right_loops.right_index:
    
    loops_i = loops.query('right_index == @i')[['chrom', 'start1', 'end1', 'left_index']]
    left_loop_BED = BedTool.from_dataframe(loops_i)

    if len(left_loop_BED) > 1:
        left_loop_keep = (left_loop_BED
                           .sort()
                           .merge(d = 10000, c = 4, o = 'collapse')
                           .to_dataframe()
                           .rename(columns = {'name':'left_index'}))
        
        if len(left_loop_keep) < len(loops_i):
            
            left_loop_keep['left_index'] = left_loop_keep['left_index'].str.split(',')
            left_loop_keep = left_loop_keep.explode('left_index')

            for ii in left_loop_keep.left_index.unique():

                loops.loc[(loops.right_index == i) & 
                          (loops.left_index == int(ii)),
                          'start1b'] = int(left_loop_keep.loc[left_loop_keep.left_index == ii,'start'])
                loops.loc[(loops.right_index == i) & 
                          (loops.left_index == int(ii)),
                          'end1b'] = int(left_loop_keep.loc[left_loop_keep.left_index == ii,'end'])

In [12]:
# Fill in the coordinates of anchors that don't change

loops.loc[np.isnan(loops.start1b), "start1b"] = loops.loc[np.isnan(loops.start1b), "start1"]
loops.loc[np.isnan(loops.end1b), "end1b"] = loops.loc[np.isnan(loops.end1b), "end1"]

loops.loc[np.isnan(loops.start2b), "start2b"] = loops.loc[np.isnan(loops.start2b), "start2"]
loops.loc[np.isnan(loops.end2b), "end2b"] = loops.loc[np.isnan(loops.end2b), "end2"]

In [ ]:
# Save the new combined anchors

before = len(loops)
loops = loops[['chrom', 'start1b', 'end1b', 'start2b', 'end2b']].drop_duplicates().rename(columns = {'start1b':'start1', 'end1b':'end1',
                                                                                                      'start2b':'start2', 'end2b':'end2'})

loops['Index'] = loops.index

print((1 - len(loops)/before*100), "% of loops were combined")

In [15]:
# Get window to look for variants so that the E-P loop and variant can fit in the ~1Mb predictive window

dist_cutoff = 900000

center_coord1 = [(x+y)/2 for x,y in zip(loops.start1, loops.end1)]
center_coord2 = [(x+y)/2 for x,y in zip(loops.start2, loops.end2)]

remaining_dist = [dist_cutoff - abs(x-y) for x,y in zip(center_coord1, center_coord2)]

# left and right most positions for variant to be in
loops['left_cutoff'] = [int(x - y) for x,y in zip(center_coord1, remaining_dist)]
loops.loc[loops.left_cutoff < 1,'left_cutoff'] = 1
loops['right_cutoff'] = [int(x + y) for x,y in zip(center_coord2, remaining_dist)]


In [34]:
# Get loops that overlap a promoter
E_or_P = pd.concat(
            [loops[['chrom', 'start1', 'end1', 'Index']].rename(columns = {'start1':'start', 'end1':'end'}),
            loops[['chrom', 'start2', 'end2', 'Index']].rename(columns = {'start2':'start', 'end2':'end'})],
            axis = 0)

E_or_P.start = E_or_P.start.astype('int')
E_or_P.end = E_or_P.end.astype('int')

E_or_P_BED = BedTool.from_dataframe(E_or_P)

promoter_loops = (E_or_P_BED
                  .intersect(promoter_annot_PC_BED, wa = True, wb = True)
                  .to_dataframe()
                  .rename(columns = {'name':'loop_index', 'thickEnd':'gene'})
                  [['loop_index', 'gene']])

before = len(loops)

# Only keep promoter loops
loops = loops[[x in promoter_loops.loop_index for x in loops.Index]]

print(len(loops)/before*100, '% of loops have an anchor at a promoter')

69.00785530398046 % of loops have an anchor at a promoter


In [37]:
# Get expression data

tpm_cutoff = 0.5

before = len(eN_exp)

# Get only expressed genes
eN_exp = eN_exp[eN_exp.TPM >= tpm_cutoff]

print((before - len(eN_exp))/before*100, '% of genes are not expressed')

69.78465563837295 % of genes are not expressed


In [38]:
#Filter only loops that overlap promoters of expressed genes
before = len(loops)

loops_to_keep = promoter_loops.loop_index[[x in eN_exp.gene.values for x in promoter_loops.gene]].unique()

loops = loops[[x in loops_to_keep for x in loops.Index]]

print((before - len(loops))/before*100, '% of loops are on promoters that are not expressed')

34.111344939735986 % of loops are on promoters that are not expressed


In [42]:
# Get E-Ps where at least one is accessible
E_or_P = pd.concat(
            [loops[['chrom', 'start1', 'end1', 'Index']].rename(columns = {'start1':'start', 'end1':'end'}),
            loops[['chrom', 'start2', 'end2', 'Index']].rename(columns = {'start2':'start', 'end2':'end'})],
            axis = 0)

E_or_P.start = E_or_P.start.astype('int')
E_or_P.end = E_or_P.end.astype('int')

E_or_P_BED = BedTool.from_dataframe(E_or_P)

accessible_loops = (E_or_P_BED
                  .intersect(ATAC_BED, wa = True, wb = True)
                  .to_dataframe()
                  .rename(columns = {'name':'loop_index'})
                  [['loop_index']])

before = len(loops)

# Only keep accessible loops
loops = loops[[x in accessible_loops.loop_index for x in loops.Index]]

print((before - len(loops))/before*100, '% of loops are not accessible')

0.0 % of loops are not accessible


In [ ]:
# find loops near variants
loops_windows_BED = BedTool.from_dataframe(loops[['chrom', 'left_cutoff', 'right_cutoff', 'Index']])

loop_var_pairs = (loops_windows_BED
                  .intersect(dnSVs_BED, wa = True, wb = True)
                  .to_dataframe()
                  .rename(columns = {'name':'loop_index', 'thickEnd':'var_index'})
                  [['loop_index', 'var_index']])

loop_var_pairs = loop_var_pairs.drop_duplicates() # If there are duplicates, they would be removed in the next step

In [ ]:
# Get E/Ps that each variant overlaps
E_or_P = pd.concat(
            [loops[['chrom', 'start1', 'end1', 'Index']].rename(columns = {'start1':'start', 'end1':'end'}),
            loops[['chrom', 'start2', 'end2', 'Index']].rename(columns = {'start2':'start', 'end2':'end'})],
            axis = 0)

E_or_P_BED = BedTool.from_dataframe(E_or_P)

var_on_E_or_P = (E_or_P_BED
                  .intersect(SSC_SV_results_BED, wa = True, wb = True)
                  .to_dataframe()
                  .rename(columns = {'name':'loop_index', 'thickEnd':'var_index'})
                  [['loop_index', 'var_index']])

before = len(loop_var_pairs)

# Filter out variants that overlap with E or P of loop they're in a trio with
loop_var_pairs = pd.concat([loop_var_pairs, var_on_E_or_P, var_on_E_or_P]).drop_duplicates(keep=False)

print(len(loop_var_pairs)/before*100, '% of E-P-variant trios: variant doesnt overlap E/P')

In [ ]:
# Get all loop and var info in one
loop_var_coord = loop_var_pairs.copy()

# Add info on loops to loop-variant pairs
loop_var_coord = (loops
                  .rename(columns = {'Index':'loop_index'})
                  .drop(['left_cutoff', 'right_cutoff'], axis = 1)
                  .merge(loop_var_coord, how = 'right')
                 )

# Add info on variants to loop-variant pairs
loop_var_coord = (SSC_SV_results
                  .drop('SSI', axis = 1)
                  .rename(columns = {'index':'var_index', 'SSI_matrix':'SSI'})
                  [['chrom', 'pos', 'End', 'svlen', 'svtype', 'role', 'var_index', 'MSE', 'correlation', 'SSI']]
                  .merge(loop_var_coord, how = 'right')
                 )

loop_var_coord.insert(6, 'center_coord_var', [round((x+y)/2) for x,y in zip(loop_var_coord.End, loop_var_coord.pos)])

# Get leftmost and rightmost coordinates of E, P and variant
loop_var_coord['start'] = [min(x,y) for x,y in zip(loop_var_coord.start1, 
                                                    loop_var_coord.pos)]

loop_var_coord['end'] = [max(x,y) for x,y in zip(loop_var_coord.end2, 
                                                    loop_var_coord.End)]


In [ ]:
# For duplications, center the duplicated sequence not the reference sequence
# Extend the variant coordinates on the opposite side of the loop

# Annotate variants based on whether they are on the left or right of the loop
loop_var_coord.loc[loop_var_coord.pos < loop_var_coord.start1,'var_position'] = 'left'
loop_var_coord.loc[loop_var_coord.pos > loop_var_coord.end2,'var_position'] = 'right'

# Var to the left of the loop
loop_var_coord.loc[(loop_var_coord.svtype == 'DUP') & 
                   (loop_var_coord.var_position == 'left'),
                   'start'] -= loop_var_coord.loc[(loop_var_coord.svtype == 'DUP') & 
                                                  (loop_var_coord.var_position == 'left'),
                                                   'svlen']

# Var to the right of the loop
loop_var_coord.loc[(loop_var_coord.svtype == 'DUP') & 
                   (loop_var_coord.var_position == 'right'),
                   'end'] += loop_var_coord.loc[(loop_var_coord.svtype == 'DUP') & 
                                                (loop_var_coord.var_position == 'right'),
                                                 'svlen']

# Remove trios that exceed prediction window
loop_var_coord = loop_var_coord[loop_var_coord.end - loop_var_coord.start < MB - (pixel_size*64)]


In [ ]:
# For duplications, the alternate allele should be able to fit with the loop in the predictive window
loop_var_coord.loc[(loop_var_coord.svtype == 'DUP') & 
                   (loop_var_coord.end - loop_var_coord.start + loop_var_coord.svlen > MB-(pixel_size*64)),
                   'remove'] = 'yes'
loop_var_coord = loop_var_coord[loop_var_coord.remove != 'yes'].drop('remove', axis = 1)


In [ ]:
# % of variants in predictive window with loop
len(loop_var_pairs.var_index.unique())/len(SSC_SV_results)*100

### Get variant-specific shift values

In [ ]:

# Add shift value

# when variant is left of the loop
loop_var_coord.loc[(loop_var_coord.var_position == 'left'),
                   'shift_by'] = [round((loop_end-var_end)/2) for loop_end,var_end in 
                               zip(loop_var_coord[(loop_var_coord.var_position == 'left')].end2, 
                                   loop_var_coord[(loop_var_coord.var_position == 'left')].End)]

# For duplications, consider the variant as the duplicated sequence so it's not out of the window after shifting
loop_var_coord.loc[(loop_var_coord.var_position == 'left') & (loop_var_coord.svtype == 'DUP'),
                   'shift_by'] = [round((loop_end-var_end)/2+svlen/2) for loop_end,var_end,svlen in 
                               zip(loop_var_coord[(loop_var_coord.var_position == 'left') & (loop_var_coord.svtype == 'DUP')].end2, 
                                   loop_var_coord[(loop_var_coord.var_position == 'left') & (loop_var_coord.svtype == 'DUP')].End,
                                   loop_var_coord[(loop_var_coord.var_position == 'left') & (loop_var_coord.svtype == 'DUP')].svlen)]


# when variant is right of the loop
loop_var_coord.loc[(loop_var_coord.var_position == 'right'),
                   'shift_by'] = [-round((var_start-loop_start)/2) for var_start,loop_start in 
                               zip(loop_var_coord[(loop_var_coord.var_position == 'right')].pos, 
                                   loop_var_coord[(loop_var_coord.var_position == 'right')].start1)]

# For duplications, consider the variant as the duplicated sequence so it's not out of the window after shifting
loop_var_coord.loc[(loop_var_coord.var_position == 'right') & (loop_var_coord.svtype == 'DUP'),
                   'shift_by'] = [-round((var_start-loop_start)/2+svlen/2) for var_start,loop_start,svlen in 
                               zip(loop_var_coord[(loop_var_coord.var_position == 'right') & (loop_var_coord.svtype == 'DUP')].pos, 
                                   loop_var_coord[(loop_var_coord.var_position == 'right') & (loop_var_coord.svtype == 'DUP')].start1,
                                   loop_var_coord[(loop_var_coord.var_position == 'right') & (loop_var_coord.svtype == 'DUP')].svlen)]




In [ ]:
# Get input files for SuPreMo

for_supremo = loop_var_coord[~np.isnan(loop_var_coord.shift_by)][['chrom', 'pos', 'End', 'svtype', 'svlen', 'shift_by']]
for_supremo.insert(2, 'REF', '-')
for_supremo.insert(3, 'ALT', '-')
for_supremo.columns = ['CHROM', 'POS', 'REF', 'ALT', 'END', 'SVTYPE', 'SVLEN', 'shift_by']

# input file
for_supremo.drop('shift_by', axis = 1).to_csv(f'{repo_dir}data/EP_for_SuPreMo.txt', 
                                           sep = '\t', index = False)

# shift file
for_supremo[['shift_by']].to_csv(f'{repo_dir}data/EP_for_SuPreMo_shifts.txt', 
                                 sep = '\t', index = False)

# Weights file
weights_file = pd.concat([loop_var_coord[['chrom', 'start1', 'end1']]
                          .rename(columns = {'chrom':'chr', 'start1':'start', 'end1':'end'}),
                          loop_var_coord[['chrom', 'start2', 'end2']]
                          .rename(columns = {'chrom':'chr', 'start2':'start', 'end2':'end'})],
                         axis = 0).reset_index(drop = True)
weights_file.to_csv(f'{repo_dir}data/EP_for_SuPreMo_weights.txt', 
                    sep = '\t', index = False)


In [ ]:
# Get output

input_file = pd.read_csv(f'{repo_dir}data/EP_for_SuPreMo_full.txt', sep = '\t')

supremo_scores = pd.concat([input_file.reset_index(drop = True),
                            pd.read_csv(f'{repo_dir}data/supremo-akita_output_weighted/', 
                                        sep = '\t').drop('var_index', axis = 1)],
                           axis = 1)
for corr_score in ['correlation', 'corr_HFF_shifted']:
    supremo_scores.loc[:,corr_score] = [1-x for x in supremo_scores[corr_score]]

supremo_scores.to_csv(f'{repo_dir}data/EP_scores_eN_masked_SuPreMo_240913', 
                      sep = '\t', index = False)